# Import relevant modules

In [1]:
import numpy as np
import pandas as pd
import random
import datetime
import names
import ccard

# Generate relevant tables with sample data

### Generate table with room data

Set up a DataFrame 'room_data' to hold room-specific info.

In [2]:
# Create index and column names
room_names = ['Room_{}'.format(i+1) for i in np.arange(18)]
room_details = [
    'adult_capacity', 
    'child_capacity', 
    'base_cost', 
    'child_cot_cost', 
    'view_type'
    ]

# Construct empty DataFrame
room_data = pd.DataFrame(
    index = room_names, 
    columns = room_details
    )
room_data.index.name = 'room_name'

Populate room_data with sample data

In [3]:
# Set basic data
room_data.loc[:, 'adult_capacity'] = 2
room_data.loc[:, 'child_capacity'] = 2
room_data.loc['Room_1':'Room_9', 'view_type'] = 'garden'
room_data.loc['Room_10':'Room_18', 'view_type'] = 'city'
room_data.loc[:, 'child_cot_cost'] = 20

# Set different costs for garden and city views
room_data.loc[room_data['view_type'] == 'garden', 'base_cost'] = 100
room_data.loc[room_data['view_type'] == 'city', 'base_cost'] = 80

In [4]:
# Export table to ...

In [5]:
# Tests/validations?
# make sure all view_types in ['garden', 'city']
# Check final prices are in the 50-200 range
# Check total capacities are <6

### Generate table with customer data

Set up table cust_data to hold customer-specific info.

In [6]:
# Create index and column names
cust_names = list(set(names.get_full_name() for i in range(500)))
cust_details = [
    'customer_initials',
    'total_visits',
    'credit_card_type',
    'credit_card_number',
    'email',
    'phone_number'
    ]

# Construct empty DataFrame
cust_data = pd.DataFrame(
    index = cust_names,
    columns = cust_details
    )
cust_data.index.name = 'customer_name'

Populate cust_data with sample data

In [7]:
# Use index to figure out customer initials
cust_data.loc[:, 'customer_initials'] = list(
    map(
        lambda x: '{}'.format(''.join([i[0].upper() for i in x.split()])), 
        cust_data.index
        )
    )

# Assign a random number of total visits between 1 and 3
cust_data.loc[:, 'total_visits'] = np.random.randint(
    low = 1, 
    high = 3, 
    size = cust_data.shape[0]
    )

# Generate sample emails based on customer names
cust_data.loc[:, 'email'] = list(
    map(
        lambda x: '{}@gmail.com'.format('.'.join([i.lower() for i in x.split()])), 
        cust_data.index
        )
    )

# Assign card type as either visa or mastercard to all customers
card_types = ['visa', 'mastercard']
cust_data.loc[:, 'credit_card_type'] = [
    np.random.choice(card_types) for i in range(cust_data.shape[0])
    ]

# Generate appropriate card numbers for customers based on card type
num_visa_cards = np.sum(cust_data['credit_card_type'] == 'visa')
num_master_cards = np.sum(cust_data['credit_card_type'] == 'mastercard')
cust_data.loc[cust_data['credit_card_type'] == 'visa', 'credit_card_number'] = [
    str(ccard.visa()) for i in range(num_visa_cards)
    ]
cust_data.loc[cust_data['credit_card_type'] == 'mastercard', 'credit_card_number'] = [
    str(ccard.mastercard()) for i in range(num_master_cards)
    ]

# Assign random phone numbers to customers
phone_nums = list(
    map(
        str, 
        np.random.randint(
            low = 1000000000, 
            high = 9999999999, 
            size = cust_data.shape[0]
            )
        )
    )
cust_data.loc[:, 'phone_number'] = list(
    map(
        lambda x: '{} {} {}'.format(x[0:2], x[2:6], x[6:10]), 
        phone_nums
        )
    )

In [8]:
# Export table to ...

In [9]:
# Tests/validations?
# Will need test to make sure first name and last name only entered

### Generate table with reservations data

In [10]:
# Reservation code x (staff member, customer name, num guests, reservation details..., promo code, price)
# - price is more if child

# New entry into reservation table -> generates option for return customer flag if email matches previous, or last name matches


Set up a DataFrame 'reservation_data' to hold reservation info.

In [11]:
# Create index and column names
reservation_fields = [
    'staff_member_initials',
    'customer_name',
    'arrival_date',
    'departure_date',
    'reservation_status',
    'repeat_customer_flag',
    'num_adults',
    'num_children',
    'num_infants', # no charge b/c no cots needed?
    'num_rooms_booked',
    'rooms_booked', # list of room_names
    'promo_code',
    'total_price',
    'credit_card_info',
    'staff_notes'
    ]

# Determine total number of reservations using cust_data
num_reservations = sum(cust_data.total_visits)
index_placeholder = np.arange(num_reservations)

# Construct empty DataFrame
reservation_data = pd.DataFrame(
    index = index_placeholder,
    columns = reservation_fields
    )

Populate reservation_data with sample data

In [12]:
# Assign a random staff member for each reservation
staff_members = ['BR', 'VP', 'KJ', 'JT', 'MT']
reservation_data.loc[:, 'staff_member_initials'] = [
    np.random.choice(staff_members) for i in range(reservation_data.shape[0])
    ]

# Assign a random customer for each reservation
visits_data = cust_data.total_visits
customers = sum(
    [[name] * num_visits 
     for name, num_visits 
     in zip(visits_data.index, visits_data.values)],
    []
    )
random.shuffle(customers)
reservation_data.loc[:, 'customer_name'] = customers

# Assign a random arrival date
# Reservations date range will span last 1 year to 1 month from today
today = datetime.datetime.strptime('20/09/2023', "%d/%m/%Y").date()
date_list = [today + datetime.timedelta(days = x) for x in range(-365, 31)]
reservation_data.loc[:, 'arrival_date'] = [
    np.random.choice(date_list) for i in range(reservation_data.shape[0])
    ]

# Assume stays of uniform probability between 1 and 7 days 
reservation_data.loc[:, 'departure_date'] = (
    reservation_data.loc[:, 'arrival_date']
    + [datetime.timedelta(days = np.random.randint(low = 1, high = 7)) 
       for i in range(reservation_data.shape[0])]
    )

# Generate reservation statuses based on dates given
# (cancelled/upcoming/active/completed)
reservation_data.loc[:, 'reservation_status'] = 'active'
reservation_data.loc[reservation_data['arrival_date'] > today, 'reservation_status'] = [
    'upcoming']
reservation_data.loc[reservation_data['departure_date'] < today, 'reservation_status'] = [
    'completed']
# Assume 1% of reservations get cancelled
reservation_data.loc[reservation_data.index % 100 == 0, 'reservation_status'] = [
    'cancelled']

# Assign a random flag for repeat customer Y/N
repeat_flags = ['Y', 'N']
reservation_data.loc[:, 'repeat_customer_flag'] = [
    np.random.choice(repeat_flags) for i in range(reservation_data.shape[0])
    ]

# Assign a random number of adults between 1 & 4
num_adults = [1, 2, 3, 4]
reservation_data.loc[:, 'num_adults'] = [
    np.random.choice(num_adults) for i in range(reservation_data.shape[0])
    ]

# Assign a random number of children between 0 & 2
num_children = [0, 1, 2]
reservation_data.loc[:, 'num_children'] = [
    np.random.choice(num_children) for i in range(reservation_data.shape[0])
    ]

# Assign a random number of infants between 0 & 2
num_infants = [0, 1, 2]
reservation_data.loc[:, 'num_infants'] = [
    np.random.choice(num_infants) for i in range(reservation_data.shape[0])
    ]

# Calculate the number of rooms booked based on reservation size
reservation_data.loc[:, 'num_rooms_booked'] = 1
reservation_data.loc[reservation_data['num_adults'] > 2, 'num_rooms_booked'] = 2

# Assign specific rooms randomly based on number of rooms booked
reservation_data.loc[:, 'rooms_booked'] = 1

# Remove double-bookings in sample data


In [23]:
reservation_data.loc[:,['arrival_date','num_rooms_booked']].groupby('arrival_date').sum()


,num_rooms_booked
arrival_date,
2022-09-20,4
2022-09-22,3
2022-09-23,2
2022-09-24,4
2022-09-25,3
...,...
2023-10-15,6
2023-10-16,3
2023-10-18,2


In [14]:
reservation_data.head(10)

,staff_member_initials,customer_name,arrival_date,departure_date,reservation_status,repeat_customer_flag,num_adults,num_children,num_infants,num_rooms_booked,rooms_booked,promo_code,total_price,credit_card_info,staff_notes
0,MT,Marion Hodges,2023-04-19,2023-04-24,cancelled,Y,4,2,2,2,NaN,NaN,NaN,NaN,NaN
1,VP,Charles Tipton,2023-01-21,2023-01-24,completed,Y,2,1,2,1,NaN,NaN,NaN,NaN,NaN
2,BR,Karen Polhemus,2022-11-10,2022-11-13,completed,Y,2,0,2,1,NaN,NaN,NaN,NaN,NaN
3,VP,Annie Waldvogel,2023-03-06,2023-03-08,completed,N,1,1,1,1,NaN,NaN,NaN,NaN,NaN
4,JT,Kevin Lolley,2022-12-06,2022-12-09,completed,N,4,2,2,2,NaN,NaN,NaN,NaN,NaN
5,KJ,Luis Ramos,2022-10-11,2022-10-13,completed,N,3,0,1,2,NaN,NaN,NaN,NaN,NaN
6,JT,Laura Kelley,2023-05-14,2023-05-15,completed,Y,2,2,2,1,NaN,NaN,NaN,NaN,NaN
7,VP,Monica Alvarez,2022-12-26,2022-12-31,completed,N,1,2,1,1,NaN,NaN,NaN,NaN,NaN
8,MT,Richard James,2023-09-23,2023-09-25,upcoming,N,3,2,0,2,NaN,NaN,NaN,NaN,NaN
9,VP,Richard Jeffries,2023-02-27,2023-03-02,completed,Y,3,0,0,2,NaN,NaN,NaN,NaN,NaN


In [15]:
# code: agentinitials-dateofarrival-dateofdeparture-roomno-custinitials-sequentialno

In [16]:
# Export table to ...

In [17]:
# Tests/validations?
# Check final prices are in the 50-200 range
# Check total capacities are <6

### Generate table with room availability data
(populated with reservation code or available)

Set up a DataFrame 'availability_data' to hold room availability info.

In [18]:
# Sample date range will span last 1 year to 1 year from today
today = datetime.datetime.today().date()

# Create index and column names
date_list = [today + datetime.timedelta(days=x) for x in range(-365, 365)]

# Construct empty DataFrame
availability_data = pd.DataFrame(
    index = room_names, 
    columns = date_list,
    data = 'empty'
    )

Populate availability_data with sample reservation codes

In [19]:
# Assign reservation codes based on reservation_data

In [20]:
# Export table to ...

In [21]:
# Tests/validations?
# Check columns are valid dates?